In [4]:
import cv2
import numpy as np
import pandas as pd

# Define the colors for each ball
colors = {
    'yellow': ([20, 100, 100], [30, 255, 255]),
    'green': ([50, 100, 100], [70, 255, 255]),
    'orange': ([5, 150, 150], [15, 255, 255]),
    'white': ([0, 0, 200], [180, 20, 255])
}

# Function to detect balls based on color
def detect_balls(frame, colors):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    balls = {}
    for color, (lower, upper) in colors.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if cnts:
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            if radius > 10:
                balls[color] = (int(x), int(y))
    return balls

# Function to determine the quadrant of a ball
def get_quadrant(x, y, width, height):
    if x < width // 2:
        if y < height // 2:
            return 1
        else:
            return 3
    else:
        if y < height // 2:
            return 2
        else:
            return 4

# Initialize video capture
cap = cv2.VideoCapture('AI Assignment video.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer for the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('processed_video.mp4', fourcc, fps, (width, height))

events = []

# Track previous quadrants of balls
prev_quadrants = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
    balls = detect_balls(frame, colors)

    for color, (x, y) in balls.items():
        quadrant = get_quadrant(x, y, width, height)
        if color in prev_quadrants:
            prev_quadrant = prev_quadrants[color]
            if prev_quadrant != quadrant:
                entry_event = f"{timestamp:.2f}, {quadrant}, {color}, Entry"
                exit_event = f"{timestamp:.2f}, {prev_quadrant}, {color}, Exit"
                events.append(entry_event)
                events.append(exit_event)
                cv2.putText(frame, f"Entry: {timestamp:.2f} s, Quadrant: {quadrant}", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(frame, f"Exit: {timestamp:.2f} s, Quadrant: {prev_quadrant}", (x, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        prev_quadrants[color] = quadrant

    # Write the frame to the output video
    out.write(frame)

# Release video capture and writer
cap.release()
out.release()

# Save to text file
with open('ball_tracking_events2.txt', 'w') as f:
    for event in events:
        f.write(event + "\n")

print("Processing complete. Check 'processed_video.mp4' and 'ball_tracking_events.txt'.")


Processing complete. Check 'processed_video.mp4' and 'ball_tracking_events.txt'.


In [3]:
import cv2
import numpy as np
import pandas as pd

# Define the colors for each ball
colors = {
    'yellow': ([20, 100, 100], [30, 255, 255]),
    'green': ([50, 100, 100], [70, 255, 255]),
    'orange': ([5, 150, 150], [15, 255, 255]),
    'white': ([0, 0, 200], [180, 20, 255])
}

# Function to detect balls based on color
def detect_balls(frame, colors):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    balls = {}
    for color, (lower, upper) in colors.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if cnts:
            c = max(cnts, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            if radius > 10:
                balls[color] = (int(x), int(y))
    return balls

# Function to determine the quadrant of a ball
def get_quadrant(x, y, width, height):
    if x < width // 2:
        if y < height // 2:
            return 1
        else:
            return 3
    else:
        if y < height // 2:
            return 2
        else:
            return 4

# Initialize video capture
cap = cv2.VideoCapture('AI Assignment video.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer for the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('processed_video.mp4', fourcc, fps, (width, height))

events = []

# Track previous quadrants of balls
prev_quadrants = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
    balls = detect_balls(frame, colors)

    for color, (x, y) in balls.items():
        quadrant = get_quadrant(x, y, width, height)
        if color in prev_quadrants:
            prev_quadrant = prev_quadrants[color]
            if prev_quadrant != quadrant:
                entry_event = f"{timestamp:.2f}, {quadrant}, {color}, Entry"
                exit_event = f"{timestamp:.2f}, {prev_quadrant}, {color}, Exit"
                events.append(entry_event)
                events.append(exit_event)
                cv2.putText(frame, f"Entry: {timestamp:.2f} s", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                cv2.putText(frame, f"Exit: {timestamp:.2f} s", (x, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        prev_quadrants[color] = quadrant

    # Write the frame to the output video
    out.write(frame)

# Release video capture and writer
cap.release()
out.release()

# Save to text file
with open('ball_tracking_events1.txt', 'w') as f:
    for event in events:
        f.write(event + "\n")

print("Processing complete. Check 'processed_video.mp4' and 'ball_tracking_events.txt'.")


Processing complete. Check 'processed_video.mp4' and 'ball_tracking_events.txt'.
